# EXAMPLE DATA

In [1]:
# market mean and std dev
mmn = 0.12
msd = 0.20

# stock std dev and alpha and correlation between stock and market
ssd = 0.30
alpha = 0.05
corr = 0.3

# risk-free rate
rf = 0.02

# CALCULATIONS

In [2]:
import numpy as np
import pandas as pd

# stock beta
beta = corr * ssd / msd

# market and stock risk premia
mprem = mmn - rf
sprem = alpha+beta*mprem
rprem = np.array([mprem,sprem])

# covariance matrix
mvar = msd**2
svar = ssd**2
cov = corr*msd*ssd
cov = np.array([[mvar,cov],[cov,svar]])

#tangency portfolio
tang = np.linalg.solve(cov,rprem)
tang = tang / np.sum(tang)
pd.Series(tang,index=['market','stock'])

market    0.784715
stock     0.215285
dtype: float64

In [ ]:
sharpe = (tang_mn - rf) / tang_sd
    if (tang > 0).sum() == 2:
        wts = np.linspace(0, 1, 201)
    elif tang["market"] < 0:
        wts = np.array(
            list(np.linspace(0, 1, 101))
            + list(np.linspace(1.01, max(2, 1 - 2 * tang["market"]), 100))
        )
    else:
        wts = np.array(
            list(np.linspace(min(-1, 2 * tang["stock"]), 0, 101))
            + list(np.linspace(0.01, 1, 100))
        )
    df = pd.DataFrame(
        dtype=float,
        index=range(201),
        columns=["stock", "market", "mean", "std", "msize"],
    )
    df["stock"] = 100 * wts
    df["market"] = 100 * (1 - wts)
    df["mean"] = [rf + w * sprem + (1 - w) * mprem for w in wts]
    df["std"] = np.sqrt(
        [w ** 2 * svar + (1 - w) ** 2 * mvar + 2 * w * (1 - w) * c for w in wts]
    )
    df.loc[200] = [stock, 1 - stock, tang_mn, tang_sd, 15]
    df = df.sort_values(by="mean")

    df2 = pd.DataFrame(dtype=float, index=range(101), columns=["mean", "std"])
    grid = np.linspace(0, 1.1 * df["std"].max(), 101)
    df2["std"] = grid
    df2["mean"] = rf + sharpe * grid
    df2["market"] = 100 * (df2["std"] / tang_sd) * tang["market"]
    df2["stock"] = 100 * (df2["std"] / tang_sd) * stock

    df3 = pd.DataFrame(
        dtype=float,
        index=["stock", "tangency", "market"],
        columns=["mean", "std", "stock", "market"],
    )
    df3.loc["stock"] = [rf + alpha + beta * mprem, ssd, 100, 0]
    df3.loc["market"] = [rf + mprem, msd, 0, 100]
    df3.loc["tangency"] = [tang_mn, tang_sd, tang["stock"], tang["market"]]

    return df, df2, df3


def figtbl(rf, mmn, msd, beta, rsd, alpha):
    df, df2, df3 = data(rf, mmn, msd, beta, rsd, alpha)
    trace1 = go.Scatter(
        x=df["std"],
        y=df["mean"],
        mode="lines",
        line=dict(color=green),
        text=df["market"],
        customdata=df["stock"],
        hovertemplate="benchmark: %{text:.1f}%<br>asset: %{customdata:.1f}%<extra></extra>",
        showlegend=False
    )
    trace2 = go.Scatter(
        x=df2["std"],
        y=df2["mean"],
        mode="lines",
        line=dict(color=blue),
        text=df2["market"],
        customdata=df2["stock"],
        hovertemplate="benchmark: %{text:.1f}%<br>asset: %{customdata:.1f}%<extra></extra>",
        showlegend=False,
    )
    trace3a = go.Scatter(
        x=[df3["std"].loc["stock"]],
        y=[df3["mean"].loc["stock"]],
        mode="markers",
        hovertemplate="asset<extra></extra>",
        marker=dict(size=15, color=red),
        name="Asset",
    )
    trace3b = go.Scatter(
        x=[df3["std"].loc["market"]],
        y=[df3["mean"].loc["market"]],
        mode="markers",
        hovertemplate="benchmark<extra></extra>",
        marker=dict(size=15, color=yellow),
        name="Benchmark"
    )
    trace3c = go.Scatter(
        x=[df3["std"].loc["tangency"]],
        y=[df3["mean"].loc["tangency"]],
        mode="markers",
        text=[df3["stock"].loc["tangency"]],
        customdata=[df3["market"].loc["tangency"]],
        hovertemplate="Tangency<br>benchmark=%{customdata:.1%}<br>asset=%{text:.1%}<extra></extra>",
        marker=dict(size=15, color=green),
        name="Tangency",
    )
    fig = go.Figure()
    for trace in [trace1, trace2, trace3a, trace3b, trace3c]:
        fig.add_trace(trace)
    fig.layout.xaxis["title"] = "Standard Deviation"
    fig.layout.yaxis["title"] = "Expected Return"
    xmax = 1.25 * np.max([df["std"].max(), df2["std"].max()])
    ymax = 1.25 * np.max([df["mean"].max(), df2["mean"].max()])
    ymin = np.min([0, 1.25 * df["mean"].min()])
    fig.update_xaxes(range=[0, xmax])
    fig.update_yaxes(range=[ymin, ymax])
    fig.update_yaxes(tickformat=".0%")
    fig.update_xaxes(tickformat=".0%")
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    return largefig(fig, showlegend=True)
